In [2]:
from helperutil import file_date_util as util
import sys
import os

# sys.path.insert(0, './dags')

# from restaurant_pipeline import default_args
# from restaurant_pipeline import get_process_file_list

In [11]:
import pandas as pd
from datetime import date as d

In [4]:
from sqlalchemy import create_engine

In [3]:
# # setup default arguments
# default_args = {
#     'download_url': 'https://data.cityofnewyork.us/resource/9w7m-hzhe.json?inspection_date=',
#     'download_date_days_ago': -3,
#     'download_date_months_ago': -3,
#     'db_hostname': '192.168.1.221',
#     'db_username': 'restuser',
#     'db_password': 'restuser',
#     'db_database': 'restaurantdb',
# }


# setup default arguments
default_args = {
    'path_name': '/opt/app/python/app/restaurant_recommendation/data',
    'pattern_name': 'restaurant_data*json',
    'download_url': 'https://data.cityofnewyork.us/resource/9w7m-hzhe.json?inspection_date=',
    'download_date_days_ago': -3,
    'download_date_months_ago': -4,
    'db_hostname': 'localhost',
    'db_username': 'root',
    'db_password': 'workadmin',
    'db_database': 'restaurant_inbox',
}



In [9]:
files_to_process = util.getdownloadedNthFiles(default_args['path_name'], default_args['pattern_name'])


In [12]:
dataframes = (pd.read_json(f, lines=True, encoding='UTF8') for f in files_to_process)
big_dataframe = pd.concat(dataframes, ignore_index=True)

/home/developer/python-dev/airflow_project1/env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [15]:
# big_dataframe.head(10)
big_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461958 entries, 0 to 461957
Data columns (total 18 columns):
action                   461958 non-null object
boro                     461958 non-null object
building                 461790 non-null object
camis                    461958 non-null int64
critical_flag            461958 non-null object
cuisine_description      461958 non-null object
dba                      461958 non-null object
grade                    239409 non-null object
grade_date               226028 non-null object
inspection_date          461958 non-null object
inspection_type          461958 non-null object
phone                    461958 non-null object
record_date              461958 non-null object
score                    439787 non-null float64
street                   461958 non-null object
violation_code           456976 non-null object
violation_description    455223 non-null object
zipcode                  461789 non-null float64
dtypes: float64(2), int6

In [54]:
df_cuisine = big_dataframe[['camis', 'cuisine_description']].sort_values('camis')
df_cuisine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 461958 entries, 406338 to 461633
Data columns (total 2 columns):
camis                  461958 non-null int64
cuisine_description    461958 non-null object
dtypes: int64(1), object(1)
memory usage: 10.6+ MB


In [68]:
df_cuisine_unique = df_cuisine.drop_duplicates()
s = df_cuisine_unique['camis']
s.value_counts()

50045337    3
50059802    3
50048150    3
41611968    3
50008878    3
50041043    3
50012165    3
50074100    2
50073904    2
50007880    2
50071401    2
50014149    2
50044450    2
50034567    2
50059059    2
50069942    2
50079762    2
50068505    2
50051529    2
40364305    2
50064635    2
41471984    2
41310209    2
50067183    2
50055447    2
50070677    2
50073048    2
41408734    2
50011013    2
50071033    2
           ..
50068330    1
50058095    1
50010356    1
41474862    1
41337645    1
50066219    1
50019058    1
50004725    1
40860410    1
40399611    1
41263873    1
41665894    1
41388804    1
50037509    1
41325319    1
50012945    1
41516864    1
41628435    1
50006804    1
50070293    1
50074391    1
50033435    1
40387357    1
41395589    1
40513885    1
50045729    1
41321253    1
50075962    1
41089834    1
41484288    1
Name: camis, Length: 28555, dtype: int64

In [81]:
df_cuisine_unique.drop_duplicates()[df_cuisine_unique['camis'] == 50045337]

,camis,cuisine_description
269697,50045337,Indian
408293,50045337,American
444443,50045337,Chicken


In [47]:
df_violation = big_dataframe.loc[:, ['critical_flag', 'camis', 'inspection_date', 'violation_code', 'violation_description']]
df_violation[df_violation['critical_flag']  == 'Critical']

,critical_flag,camis,inspection_date,violation_code,violation_description
0,Critical,41576311,2015-01-02T00:00:00.000,04M,Live roaches present in facility's food and/or...
1,Critical,41128262,2015-01-02T00:00:00.000,06D,"Food contact surface not properly washed, rins..."
2,Critical,50003001,2015-01-02T00:00:00.000,04H,"Raw, cooked or prepared food is adulterated, c..."
3,Critical,50010338,2015-01-02T00:00:00.000,06C,Food not protected from potential source of co...
6,Critical,50004144,2015-01-02T00:00:00.000,04L,Evidence of mice or live mice present in facil...
11,Critical,41222832,2015-01-02T00:00:00.000,04L,Evidence of mice or live mice present in facil...
12,Critical,41169143,2015-01-02T00:00:00.000,06C,Food not protected from potential source of co...
13,Critical,40592981,2015-01-02T00:00:00.000,02G,Cold food item held above 41Âº F (smoked fish ...
15,Critical,41488846,2015-01-02T00:00:00.000,04H,"Raw, cooked or prepared food is adulterated, c..."
19,Critical,41649180,2015-01-02T00:00:00.000,06D,"Food contact surface not properly washed, rins..."


In [27]:
# big_dataframe['critical_flag'].unique()
# array(['Critical', 'Not Critical', 'Not Applicable'], dtype=object)

# big_dataframe['inspection_type'].unique()

# big_dataframe['violation_code'].unique()
big_dataframe['violation_description'].unique()

# big_dataframe['cuisine_description'].unique()

array(["Live roaches present in facility's food and/or non-food areas.",
       'Food contact surface not properly washed, rinsed and sanitized after each use and following any activity when contamination may have occurred.',
       'Raw, cooked or prepared food is adulterated, contaminated, cross-contaminated, or not discarded in accordance with HACCP plan.',
       'Food not protected from potential source of contamination during storage, preparation, transportation, display or service.',
       'Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact surface or equipment improperly maintained and/or not properly sealed, raised, spaced or movable to allow accessibility for cleaning on all sides, above and underneath the unit.',
       'Plumbing not properly installed or maintained; anti-siphonage or backflow prevention device not provided where required; equipment or floor not properly drained; sewage disposal system in disrepair or not functioni

In [6]:
engine = create_engine("mysql+pymysql://" + default_args['db_username'] + ":" + default_args['db_password'] + "@" + default_args['db_hostname'] + "/" + default_args['db_database'] + "", echo=False)
conn = engine.connect()

In [42]:
# files_to_process = get_process_file_list()
# dataframes = (pd.read_json(f, lines=True, encoding='UTF8') for f in files_to_process)
# big_dataframe = pd.concat(dataframes, ignore_index=True)
# # big_dataframe = big_dataframe.reset_index()

sql_string = """
select distinct
v_camis as camis, v_grade as grade, v_grade_date as grade_date 
from restaurant_inbox
"""

df_grade = pd.read_sql(sql_string, conn)
big_dataframe = df_grade

In [51]:
df_grade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110415 entries, 0 to 110414
Data columns (total 3 columns):
camis         110414 non-null float64
grade         110319 non-null object
grade_date    110316 non-null object
dtypes: float64(1), object(2)
memory usage: 2.5+ MB


In [53]:
# downloading_list
# path_name

# date_end
# date_start

#big_dataframe.head(2)
big_dataframe[big_dataframe['grade_date'] != '0000-00-00 00:00:00'].head(10)

,camis,grade,grade_date
3,50041630.0,A,2018-01-31 00:00:00
4,41130552.0,A,2018-01-31 00:00:00
11,50008339.0,A,2018-01-31 00:00:00
12,50017896.0,A,2018-01-31 00:00:00
18,41717443.0,A,2018-01-31 00:00:00
21,41249587.0,A,2018-01-31 00:00:00
25,50041181.0,A,2018-01-31 00:00:00
28,50003103.0,A,2018-01-31 00:00:00
32,50004139.0,Z,2018-01-31 00:00:00
33,50053806.0,Z,2018-01-31 00:00:00


In [55]:
big_dataframe = big_dataframe[big_dataframe['grade_date'] != '0000-00-00 00:00:00']
big_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87249 entries, 3 to 110414
Data columns (total 3 columns):
camis         87248 non-null float64
grade         87153 non-null object
grade_date    87150 non-null object
dtypes: float64(1), object(2)
memory usage: 2.7+ MB


In [56]:
big_dataframe = big_dataframe[['camis', 'grade', 'grade_date']].drop_duplicates(subset=['camis', 'grade', 'grade_date'])
big_dataframe = big_dataframe[big_dataframe['grade_date'].isna() == False]
big_dataframe.columns = ['business_number', 'grade', 'grade_date']


In [57]:
#df_grade = big_dataframe
big_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87150 entries, 3 to 110414
Data columns (total 3 columns):
business_number    87150 non-null float64
grade              87150 non-null object
grade_date         87150 non-null object
dtypes: float64(1), object(2)
memory usage: 2.7+ MB


In [60]:
df_grade = big_dataframe
df_grade.head()

,business_number,grade,grade_date
3,50041630.0,A,2018-01-31
4,41130552.0,A,2018-01-31
11,50008339.0,A,2018-01-31
12,50017896.0,A,2018-01-31
18,41717443.0,A,2018-01-31


In [59]:
df_grade['grade_date'] = df_grade['grade_date'].astype('datetime64')
# s[s.isna() == True].value_counts()
# s.value_counts

In [37]:
# df_grade.head(10)
df_grade = df_grade[df_grade['grade_date'].isna() == False]

In [38]:
#df_grade.head(10)
df_grade = df_grade[df_grade['grade_date'].isna() == False]

In [62]:
df_grade[df_grade.isna() == False].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87150 entries, 3 to 110414
Data columns (total 3 columns):
business_number    87150 non-null float64
grade              87150 non-null object
grade_date         87150 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 2.7+ MB


In [63]:
truncate_sql = """
truncate table trans_dim_grade
"""
update_res = pd.io.sql.execute(truncate_sql, conn)

In [64]:
df_grade.to_sql('trans_dim_grade', con=engine, if_exists='append', index=False)

In [65]:
delete_sql = """
delete a
from trans_dim_grade a left join dim_grade b on a.business_number = b.business_number and a.grade_date = b.grade_date
where b.business_number is not null
"""
delete_res = pd.io.sql.execute(delete_sql, conn)

In [66]:
insert_sql = """
insert into dim_grade (business_number, grade, grade_date, create_date, modify_date)
select business_number, grade, grade_date, now(), now() from trans_dim_grade
"""
update_res = pd.io.sql.execute(insert_sql, conn)